# SELECCIÓN DE SEGMENTO CRITICO: MOTORES OBJETIVOS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# Ruta del archivo Excel
ruta_archivo = '/content/consolidado_motores.xlsx'

# Cargar el archivo Excel en un DataFrame
consolidado = pd.read_excel(ruta_archivo, sheet_name='Consolidado')

In [ ]:
# Filtrar el DataFrame para obtener solo los motores con voltaje entre 550 y 585
consolidado2 = consolidado[(consolidado['SEG_VOLT'] == "BAJO VOLTAJE 2(550-585)")]

In [ ]:
motores_objetivos = consolidado2[
    (consolidado['POTENCIA (HP)'] <= 100) &
    (consolidado['Polos'] == 4) ]

print(len(motores_objetivos))

700


<ipython-input-7-6f6264c63c5b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  motores_objetivos = consolidado2[


In [ ]:
print(f"Total de motores objetivos : {len(motores_objetivos)}")
print(f"Total de motores objetivos con características repetidas(unicos): {sum(len(grupo) >= 1 for _, grupo in motores_objetivos.groupby(['MARCA ', 'R.P.M', 'POTENCIA (HP)', 'FRAME']))}")


Total de motores objetivos : 700
Total de motores objetivos con características repetidas(unicos): 313


In [ ]:
#DATA MOTORES UNICOS
motores_unicos = motores_objetivos.drop_duplicates(subset=['MARCA ', 'R.P.M', 'POTENCIA (HP)', 'FRAME','Polos'])
print(f"Cantidad de motores objetivos unicos: {len(motores_unicos)}")

Cantidad de motores objetivos unicos: 313


# ID PARA MOTORES Y CANTIDADES


Crear ID para cada motor unico

In [ ]:
# AGREGAR ID A CADA MOTOR UNICO
import string

def generate_unique_id(df):
  """
  Genera un ID único para cada motor en el DataFrame.

  Args:
    df: DataFrame con motores únicos.

  Returns:
    DataFrame con una nueva columna 'ID_Motor' con IDs únicos.
  """

  df['ID_Motor'] = ['M' + str(i).zfill(3) for i in range(1, len(df) + 1)]
  return df

motores_unicos = generate_unique_id(motores_unicos)

        N° PROCESO SECTOR/UBICACIÓN                EQUIPO           MARCA   \
0        1  04 PPC     FILTROS MOLY     FILTRO TAMBOR 601              WEG   
2        3  04 PPC     FILTROS MOLY  FILTRO ACCIONAMIENTO   ALLIS CHALMERS   
8        9  04 PPC     FILTROS MOLY   FILTRO TOR. SIN FIN           LENZE    
9       10  04 PPC              NaN          TORNILLO N°5           S/DATO   
10      11  04 PPC     FILTROS MOLY     BOMBA ACEITE Nº 1   ALLIS CHALMERS   
...    ...     ...              ...                   ...              ...   
1160  1162  03 PMC     PLANTA P.T.R          BOMBA Nº 773  U.S. ELECTRICAL   
1164  1166  03 PMC     PLANTA P.T.R          BOMBA Nº 793  U.S. ELECTRICAL   
1167  1169  03 PMC     PLANTA P.T.R          BOMBA Nº 750             ASEA   
1168  1170  03 PMC     PLANTA P.T.R          BOMBA Nº 751             ASEA   
1174  1176  03 PMC     PLANTA P.T.R          BOMBA Nº 786             TECO   

                   Nº SERIE   FRAME  POTENCIA (HP) VOLTAJE   R.

<ipython-input-8-d60e244aab7e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID_Motor'] = ['M' + str(i).zfill(3) for i in range(1, len(df) + 1)]


Agregar ID a motores objetivos

In [ ]:
# AGREGAR ID A MOTORES OBJETIVOS
motores_objetivos= pd.merge(motores_objetivos, motores_unicos[['MARCA ', 'R.P.M', 'POTENCIA (HP)', 'FRAME', 'ID_Motor']],
                                    on=['MARCA ', 'R.P.M', 'POTENCIA (HP)', 'FRAME'], how='left')

print(motores_objetivos)

       N° PROCESO SECTOR/UBICACIÓN                EQUIPO          MARCA   \
0       1  04 PPC     FILTROS MOLY     FILTRO TAMBOR 601             WEG   
1       3  04 PPC     FILTROS MOLY  FILTRO ACCIONAMIENTO  ALLIS CHALMERS   
2       9  04 PPC     FILTROS MOLY   FILTRO TOR. SIN FIN          LENZE    
3      10  04 PPC              NaN          TORNILLO N°5          S/DATO   
4      11  04 PPC     FILTROS MOLY     BOMBA ACEITE Nº 1  ALLIS CHALMERS   
..    ...     ...              ...                   ...             ...   
695  1170  03 PMC     PLANTA P.T.R          BOMBA Nº 751            ASEA   
696  1171  03 PMC     PLANTA P.T.R          BOMBA Nº 752             WEG   
697  1172  03 PMC     PLANTA P.T.R          BOMBA Nº 756          BALDOR   
698  1174  03 PMC     PLANTA P.T.R          BOMBA Nº 758  ALLIS CHALMERS   
699  1176  03 PMC     PLANTA P.T.R          BOMBA Nº 786            TECO   

                Nº SERIE   FRAME  POTENCIA (HP) VOLTAJE   R.P.M  \
0                AS3

Agregar cantidad de motores a data de motores unicos

In [ ]:
# Contar la cantidad de motores para cada motor único
conteo_motores = motores_objetivos.groupby('ID_Motor')['ID_Motor'].count().reset_index(name='CANTIDAD')

# Unir la información del conteo al DataFrame de motores únicos
motores_unicos = pd.merge(motores_unicos, conteo_motores, on='ID_Motor', how='left')

Motores unicos con columnas reducidas (para descargar columnas realmente importantes)

In [ ]:
motores_unicos_reducidos = motores_unicos[['ID_Motor', 'R.P.M', 'POTENCIA (HP)', 'VOLTAJE', 'FRAME', 'MARCA ', 'CANTIDAD' ]]
print(motores_unicos_reducidos)

    ID_Motor   R.P.M  POTENCIA (HP) VOLTAJE   FRAME           MARCA   CANTIDAD
0       M001  1480.0            3.0     575   184-T              WEG         1
1       M002  1450.0            1.0     575   145-T   ALLIS CHALMERS         1
2       M003  1420.0            3.0     575  S/DATO           LENZE          1
3       M004  1455.0            3.0     575   132-M           S/DATO         1
4       M005  1450.0           10.0     550   254-T   ALLIS CHALMERS         4
..       ...     ...            ...     ...     ...              ...       ...
308     M309  1475.0           60.0     575   365-T  U.S. ELECTRICAL         4
309     M310  1475.0           40.0     575   326-T  U.S. ELECTRICAL         1
310     M311  1470.0           13.0     575  S/DATO             ASEA         1
311     M312  1470.0           30.0     575  S/DATO             ASEA         1
312     M313  1455.0           40.0     575   225-S             TECO         1

[313 rows x 7 columns]


In [ ]:
motores_unicos_reducidos.to_excel('Motores_Unicos.xlsx', index=False)

# Download the Excel file
from google.colab import files
files.download('Motores_unicos.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>